# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Autumn 2025** </center>
---
### <center> **Examples on Machine Learning: Logistic Regression** </center>
---
**Profesor**: Pablo Camarillo Ramirez

## Recreate docker images and spark cluster:

Go to **spark** directory and run:
     
     sh build-images.sh
     docker compose up --scale spark-worker=1 -d

# Create SparkSession

In [28]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ML: Logistic Regression") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 38532)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/opt/spark/python/pyspark/accumulators.py", line 299, in handle
    poll(accum_updates)
  File "/opt/spark/python/pyspark/accumulators.py", line 271, in poll
    if self.rfile in r and func():
  File "/opt/spark/python/pyspark/accumulators.py", line 275, in accum_updates
    num_updates = read_int(self.rfile)
  File "/opt/spark/python/pyspark/serializers.py", lin

# Collect Data

In [29]:
from pcamarillor.spark_utils import SparkUtils
# Create a small dataset as a list of tuples
# Format: (label, feature_x1, feature_x2)
data = [
    (1.0, 2.0, 3.0),
    (0.0, 1.0, 2.5),
    (1.0, 3.0, 5.0),
    (0.0, 0.5, 1.0),
    (1.0, 4.0, 6.0)
]

# Define schema for the DataFrame
schema = SparkUtils.generate_schema([("label", "float"), 
                                     ("feature_x1", "float"),
                                     ("feature_x2", "float")])

# Convert list to a DataFrame
df = spark.createDataFrame(data, schema)

### Assemble the features into a single vector column

In [30]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["feature_x1", "feature_x2"], outputCol="features")
data_with_features = assembler.transform(df).select("label", "features")
data_with_features.printSchema()                                   

root
 |-- label: float (nullable = true)
 |-- features: vector (nullable = true)



# Data splitting
#### 80% training data and 20% testing data

In [44]:
train_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=57)

### Show dataset (for debugging)

In [45]:
print("Original Dataset")
df.show()

# Print train dataset
print("train set")
train_df.show()

Original Dataset
+-----+----------+----------+
|label|feature_x1|feature_x2|
+-----+----------+----------+
|  1.0|       2.0|       3.0|
|  0.0|       1.0|       2.5|
|  1.0|       3.0|       5.0|
|  0.0|       0.5|       1.0|
|  1.0|       4.0|       6.0|
+-----+----------+----------+

train set
+-----+---------+
|label| features|
+-----+---------+
|  0.0|[1.0,2.5]|
|  1.0|[2.0,3.0]|
|  0.0|[0.5,1.0]|
|  1.0|[4.0,6.0]|
+-----+---------+



# Create ML Model

In [46]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.01)

# Train ML Model

In [47]:
lr_model = lr.fit(train_df)

# Print coefficients
print("Coefficients: " + str(lr_model.coefficients))

# Display model summary
training_summary = lr_model.summary

Coefficients: [2.346116998875653,0.7963873036415707]


## Predictions

In [48]:
# Use the trained model to make predictions on the test data
predictions = lr_model.transform(test_df)

# Show predictions
predictions.select("features", "prediction", "probability").show()

+---------+----------+--------------------+
| features|prediction|         probability|
+---------+----------+--------------------+
|[3.0,5.0]|       1.0|[0.00524886113385...|
+---------+----------+--------------------+



# Test ML Model

In [49]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label",
                            predictionCol="prediction")

accuracy = evaluator.evaluate(predictions, 
                  {evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy}")
precision = evaluator.evaluate(predictions,
                  {evaluator.metricName: "weightedPrecision"})
print(f"Precision: {precision}")
recall = evaluator.evaluate(predictions,
                  {evaluator.metricName: "weightedRecall"})
print(f"Recall: {recall}")
f1 = evaluator.evaluate(predictions,
                {evaluator.metricName: "f1"})
print(f"F1 Score: {f1}")  

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


# Lab 11: Logistic regression To predict heart disease

# Data collection

In [50]:
# Define schema for the DataFrame
heart_schema = SparkUtils.generate_schema([
    ("male", "int"), 
    ("age", "int"), 
    ("education", "int"), 
    ("currentSmoker", "int"), 
    ("cigsPerDay", "int"), 
    ("BPMeds", "int"), 
    ("prevalentStroke", "int"), 
    ("prevalentHyp", "int"), 
    ("diabetes", "int"), 
    ("totChol", "int"), 
    ("sysBP", "float"), 
    ("diaBP", "float"), 
    ("BMI", "float"), 
    ("heartRate", "int"), 
    ("glucose", "int"), 
    ("TenYearCHD", "int")])

# Source: https://www.kaggle.com/datasets/dileep070/heart-disease-prediction-using-logistic-regression?resource=download

heart_df = spark.read \
                .option("header", "true") \
                .schema(heart_schema) \
                .csv("/opt/spark/work-dir/data/ml/logistic_regression/framingham.csv")

heart_df.printSchema()

root
 |-- male: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- education: integer (nullable = true)
 |-- currentSmoker: integer (nullable = true)
 |-- cigsPerDay: integer (nullable = true)
 |-- BPMeds: integer (nullable = true)
 |-- prevalentStroke: integer (nullable = true)
 |-- prevalentHyp: integer (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- totChol: integer (nullable = true)
 |-- sysBP: float (nullable = true)
 |-- diaBP: float (nullable = true)
 |-- BMI: float (nullable = true)
 |-- heartRate: integer (nullable = true)
 |-- glucose: integer (nullable = true)
 |-- TenYearCHD: integer (nullable = true)



In [51]:
heart_df.show(2)

+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|male|age|education|currentSmoker|cigsPerDay|BPMeds|prevalentStroke|prevalentHyp|diabetes|totChol|sysBP|diaBP|  BMI|heartRate|glucose|TenYearCHD|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
|   1| 39|        4|            0|         0|     0|              0|           0|       0|    195|106.0| 70.0|26.97|       80|     77|         0|
|   0| 46|        2|            0|         0|     0|              0|           0|       0|    250|121.0| 81.0|28.73|       95|     76|         0|
+----+---+---------+-------------+----------+------+---------------+------------+--------+-------+-----+-----+-----+---------+-------+----------+
only showing top 2 rows


25/10/22 01:15:22 WARN StandaloneAppClient$ClientEndpoint: Connection to 4d262414554f:7077 failed; waiting for master to reconnect...
25/10/22 01:15:22 WARN StandaloneSchedulerBackend: Disconnected from Spark cluster! Waiting for reconnection...
25/10/22 01:15:22 ERROR TransportRequestHandler: Error sending result RpcResponse[requestId=5327483500517964412,body=NioManagedBuffer[buf=java.nio.HeapByteBuffer[pos=0 lim=81 cap=156]]] to /172.24.0.4:33718; closing connection
io.netty.channel.StacklessClosedChannelException
	at io.netty.channel.AbstractChannel$AbstractUnsafe.write(Object, ChannelPromise)(Unknown Source)


# Data Splitting

# Create ML Model

# Train ML Model

# Test ML Model

In [ ]:
sc.stop()